In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

__file__ = '/content/drive/Othercomputers/MacBook_Air/path-finding-rl/data'

Mounted at /content/drive


In [5]:
from string import ascii_uppercase
#from draw_utils import *
#from pyglet.gl import *
import numpy as np
import pandas as pd
import os
import random
from datetime import datetime
import pytz
import matplotlib.pyplot as plt

# reward
move_reward = -0.1
obs_reward = -0.1
goal_reward = 10
###################################
# train or test 모드 지정
train_mode = True
###################################
print('reward:' , move_reward, obs_reward, goal_reward)

#__file__ = '/home/ogangza/heung_path_finding/path-finding-rl/data' # GCP용
local_path = os.path.abspath(os.path.join(os.path.dirname(__file__)))


class Simulator:
    def __init__(self):
        '''
        height : 그리드 높이
        width : 그리드 너비 
        inds : A ~ Q alphabet list
        '''
        #######################################################################################
        # Load train or test data
        if train_mode:  # 훈련 데이타 읽기
            self.files = pd.read_csv(os.path.join(local_path, "data/factory_order_train.csv"))
            print('data/factory_order_train.csv used')
        else:  # 테스트 데이터 읽기
            self.files = pd.read_csv(os.path.join(local_path, "data/factory_order_test.csv"))
            print('data/factory_order_test.csv used')
        #######################################################################################        
        self.height = 10
        self.width = 9
        self.inds = list(ascii_uppercase)[:17]

    def set_box(self):
        '''
        아이템들이 있을 위치를 미리 정해놓고 그 위치 좌표들에 아이템이 들어올 수 있으므로 그리드에 100으로 표시한다.
        데이터 파일에서 이번 에피소드 아이템 정보를 받아 가져와야 할 아이템이 있는 좌표만 -100으로 표시한다.
        self.local_target에 에이전트가 이번에 방문해야할 좌표들을 저장한다.
        따라서 가져와야하는 아이템 좌표와 end point 좌표(처음 시작했던 좌표로 돌아와야하므로)가 들어가게 된다.
        '''
        box_data = pd.read_csv(os.path.join(local_path, "./data/box.csv"))
        
        # 물건이 들어있을 수 있는 경우
        for box in box_data.itertuples(index = True, name ='Pandas'):
            self.grid[getattr(box, "row")][getattr(box, "col")] = 100

        # 물건이 실제 들어있는 경우
        order_item = list(set(self.inds) & set(self.items))
        order_csv = box_data[box_data['item'].isin(order_item)]

        for order_box in order_csv.itertuples(index = True, name ='Pandas'):
            self.grid[getattr(order_box, "row")][getattr(order_box, "col")] = -100
            # local target에 가야 할 위치 좌표 넣기
            self.local_target.append(
                [getattr(order_box, "row"),
                 getattr(order_box, "col")]
                )
        # self.local_target.sort() 
        self.local_target.append([9,4]) 

        # 알파벳을 Grid에 넣어서 -> grid에 2Dconv 적용 가능

    def set_obstacle(self):
        '''
        장애물이 있어야하는 위치는 미리 obstacles.csv에 정의되어 있다. 이 좌표들을 0으로 표시한다.
        '''
        obstacles_data = pd.read_csv(os.path.join(local_path, "./data/obstacles.csv"))
        for obstacle in obstacles_data.itertuples(index = True, name ='Pandas'):
            self.grid[getattr(obstacle, "row")][getattr(obstacle, "col")] = 0

    def reset(self, epi):
        '''
        reset()은 첫 스텝에서 사용되며 그리드에서 에이전트 위치가 start point에 있게 한다.

        :param epi: episode, 에피소드 마다 가져와야 할 아이템 리스트를 불러올 때 사용
        :return: 초기셋팅 된 그리드
        :rtype: numpy.ndarray
        _____________________________________________________________________________________
        items : 이번 에피소드에서 가져와야하는 아이템들
        terminal_location : 현재 에이전트가 찾아가야하는 목적지
        local_target : 한 에피소드에서 찾아가야하는 아이템 좌표, 마지막 엔드 포인트 등의 위치좌표들
        actions: visualization을 위해 에이전트 action을 저장하는 리스트
        curloc : 현재 위치
        '''

        # initial episode parameter setting
        self.epi = epi
        self.items = list(self.files.iloc[self.epi])[0]
        self.cumulative_reward = 0
        self.terminal_location = None
        self.local_target = []
        self.actions = []

        # initial grid setting
        self.grid = np.ones((self.height, self.width), dtype="float16")

        # set information about the gridworld
        self.set_box()
        self.set_obstacle()

        # start point를 grid에 표시
        ################################################
        self.curloc = [9, 4]  # 끝에 경로 길이 추가
        ################################################
        self.grid[int(self.curloc[0])][int(self.curloc[1])] = -5
        self.done = False

        return self.grid

    def apply_action(self, action, cur_x, cur_y):
        '''
        에이전트가 행한 action대로 현 에이전트의 위치좌표를 바꾼다.
        action은 discrete하며 4가지 up,down,left,right으로 정의된다.
        
        :param x: 에이전트의 현재 x 좌표
        :param y: 에이전트의 현재 y 좌표
        :return: action에 따라 변한 에이전트의 x 좌표, y 좌표
        :rtype: int, int
        '''
        new_x = cur_x
        new_y = cur_y
        # up
        if action == 0:
            new_x = cur_x - 1
        # down
        elif action == 1:
            new_x = cur_x + 1
        # left
        elif action == 2:
            new_y = cur_y - 1
        # right
        else:
            new_y = cur_y + 1

        return int(new_x), int(new_y)

    def get_reward(self, new_x, new_y, out_of_boundary):
        '''
        get_reward함수는 리워드를 계산하는 함수이며, 상황에 따라 에이전트가 action을 옳게 했는지 판단하는 지표가 된다.

        :param new_x: action에 따른 에이전트 새로운 위치좌표 x
        :param new_y: action에 따른 에이전트 새로운 위치좌표 y
        :param out_of_boundary: 에이전트 위치가 그리드 밖이 되지 않도록 제한
        :return: action에 따른 리워드
        :rtype: float
        '''

        # 바깥으로 나가는 경우
        if any(out_of_boundary):
            reward = obs_reward
                       
        else:
            # 장애물에 부딪히는 경우 
            if self.grid[new_x][new_y] == 0:
                reward = obs_reward  

            # 현재 목표에 도달한 경우
            elif new_x == self.terminal_location[0] and new_y == self.terminal_location[1]:
                reward = goal_reward

            # 그냥 움직이는 경우 
            else:
                reward = move_reward

        return reward

    def step(self, action):
        ''' 
        에이전트의 action에 따라 step을 진행한다.
        action에 따라 에이전트 위치를 변환하고, action에 대해 리워드를 받고, 어느 상황에 에피소드가 종료되어야 하는지 등을 판단한다.
        에이전트가 endpoint에 도착하면 gif로 에피소드에서 에이전트의 행동이 저장된다.

        :param action: 에이전트 행동
        :return:
            grid, 그리드
            reward, 리워드
            cumulative_reward, 누적 리워드
            done, 종료 여부
            goal_ob_reward, goal까지 아이템을 모두 가지고 돌아오는 finish율 계산을 위한 파라미터

        :rtype: numpy.ndarray, float, float, bool, bool/str

        (Hint : 시작 위치 (9,4)에서 up말고 다른 action은 전부 장애물이므로 action을 고정하는 것이 좋음)
        '''

        self.terminal_location = self.local_target[0]
        cur_x,cur_y = self.curloc
        #############################################################
        # cur_x, cur_y, path_length = self.curloc  # 경로 길이 추가
        #############################################################
        self.actions.append((cur_x, cur_y))
        goal_ob_reward = False
        new_x, new_y = self.apply_action(action, cur_x, cur_y)
        out_of_boundary = [new_x < 0, new_x >= self.height, new_y < 0, new_y >= self.width]

        # 바깥으로 나가는 경우 종료
        if any(out_of_boundary):
            #원 위치
            #new_x = cur_x
            #new_y = cur_y
            self.done = True
            goal_ob_reward = True
        else:
            # 장애물에 부딪히는 경우 종료
            if self.grid[new_x][new_y] == 0:
                #원 위치
                #new_x = cur_x
                #new_y = cur_y
                self.done = True
                goal_ob_reward = True

            # 현재 목표에 도달한 경우, 다음 목표 설정
            elif new_x == self.terminal_location[0] and new_y == self.terminal_location[1]:

                # end point 일 때
                if [new_x, new_y] == [9,4]:
                    self.done = True

                self.local_target.remove(self.local_target[0])
                self.grid[cur_x][cur_y] = 1
                self.grid[new_x][new_y] = -5
                goal_ob_reward = True
                # self.curloc = [new_x, new_y, path_length]
                self.curloc = [new_x, new_y]
            else:
                # 그냥 움직이는 경우 
                self.grid[cur_x][cur_y] = 1
                self.grid[new_x][new_y] = -5
                # self.curloc = [new_x,new_y, path_length]
                self.curloc = [new_x,new_y]
                
        reward = self.get_reward(new_x, new_y, out_of_boundary)
        self.cumulative_reward += reward
        ########################################################
        if self.done == True:
            goal_of_reward = 'finish'
        ########################################################
        # print(f"gird = \n{self.grid}")

        return self.grid, reward, self.cumulative_reward, self.done, goal_ob_reward

reward: -0.1 -0.1 10


In [8]:
import gym
import collections
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

#Hyperparameters
learning_rate = 0.0005
gamma         = 0.98
buffer_limit  = 50000
batch_size    = 32

class ReplayBuffer():
    def __init__(self):
        self.buffer = collections.deque(maxlen=buffer_limit)
    
    def put(self, transition):
        self.buffer.append(transition)
    
    def sample(self, n):
        mini_batch = random.sample(self.buffer, n)
        s_lst, a_lst, r_lst, s_prime_lst, done_mask_lst = [], [], [], [], []
        
        for transition in mini_batch:
            s, a, r, s_prime, done_mask = transition
            s_lst.append(s)
            a_lst.append([a])
            r_lst.append([r])
            s_prime_lst.append(s_prime)
            done_mask_lst.append([done_mask])
        ##############################################################################
        ## *_list를 np.array(*_lst)로 변경
        ##############################################################################
        return torch.tensor(np.array(s_lst), dtype=torch.float), torch.tensor(np.array(a_lst)), \
               torch.tensor(np.array(r_lst)), torch.tensor(np.array(s_prime_lst), dtype=torch.float), \
               torch.tensor(np.array(done_mask_lst))
    
    def size(self):
        return len(self.buffer)

class Qnet(nn.Module):
    def __init__(self):
        super(Qnet, self).__init__()
        ###################################################################
        self.fc1 = nn.Linear(94, 512)  # input 정보에 E(M, s, g) -> Map 90, start point 2, goal point 2
        ###################################################################
        self.fc2 = nn.Linear(512, 512)
        self.fc3 = nn.Linear(512, 4)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
      
    def sample_action(self, obs, epsilon):
        out = self.forward(obs)
        coin = random.random()
        if coin < epsilon:
            return random.randint(0,3)
        else : 
            return out.argmax().item()
    ###################################################
    def test_action(self, obs):  # Test용 action 추가
        out = self.forward(obs)
        return out.argmax().item()
    ###################################################
            
def train(q, q_target, memory, optimizer):
    for i in range(10):
        s,a,r,s_prime,done_mask = memory.sample(batch_size)

        q_out = q(s)
        q_a = q_out.gather(1,a)
        max_q_prime = q_target(s_prime).max(1)[0].unsqueeze(1)
        target = r + gamma * max_q_prime * done_mask
        loss = F.smooth_l1_loss(q_a, target)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

def main():
    tz = pytz.timezone('Asia/Seoul')

    env = Simulator()
    q = Qnet()
    q_target = Qnet()
    q_target.load_state_dict(q.state_dict())
    memory = ReplayBuffer()

    print_interval = 1000
    score = 0.0  
    optimizer = optim.Adam(q.parameters(), lr=learning_rate)
    print('len(env.files):', len(env.files))

    for n_epi in range(len(env.files)): # range()의 인수로 len(env.files) 사용하면 됨 (=39,999)
        
        epsilon = max(0.01, 0.08 - 0.01*(n_epi/200)) #Linear annealing from 8% to 1%
        env.reset(n_epi)
        print(env.reset(n_epi))

        ##################
        # path_length = len(env.local_target)
        # s_before = env.curloc
        # s_before.append(path_length)
        # s = np.array(s_before)
        # print(s_before)
        
        grid_map = env.grid.reshape(-1)
        start_point = [9,4]
        end_point = env.local_target[0]

        grid_map = np.append(grid_map, start_point)
        grid_map = np.append(grid_map, end_point)
        s = grid_map
        # print("grid_map = \n")
        # print(grid_map)
        ##################

        old_score = 0.0
        done = False

        while not done:  # OB, 장애물, 최종 목표 도달시 종료
            a = q.sample_action(torch.from_numpy(s).float(), epsilon)  # 인풋 정보에 경로 길이 추가
            obs, r, cum_reward, done, goal_ob_reward = env.step(a)
            s_prime = np.array(env.curloc)
            done_mask = 0.0 if done else 1.0
            memory.put((s,a,r/10.0,s_prime, done_mask))
            s = s_prime
            score += r
            '''
            if score > old_score:
                    cur_time = datetime.now(tz)
                    simple_cur_time = cur_time.strftime("%H:%M:%S")
                    print('▶ Episode #', n_epi, 'start time:', simple_cur_time, end='→')
                    print('Score =', score, 'appended actions =', s, end='...')
                    print('경로 길이:', len(env.actions))
            '''
            old_score = score
            #if done:
                #break
            # while loop 종료

        ############################################################################
                ## memory.size(): 2000 → 20000 → 25000 수정... 이후 훈련 시작
        ############################################################################
        if memory.size() > 25000:
            train(q, q_target, memory, optimizer)

        # print_interval (=1000) 마다 현황 디스플레이하고 q_target 업데이트 및 score 초기화
        if n_epi % print_interval == 0 and n_epi != 0:
            cur_time = datetime.now(tz)
            simple_cur_time = cur_time.strftime("%H:%M:%S")
            print('▶ Episode #', n_epi, 'start time:', simple_cur_time, end='→')

            q_target.load_state_dict(q.state_dict())

            print("n_episode :{}, score : {:.1f}, n_buffer : {}, eps : {:.1f}%".format(n_epi, score/print_interval, memory.size(), epsilon*100))
            score = 0.0
            # torch.save(q, '/content/drive/MyDrive/aiffelthon/data/model.pt')
            torch.save(q, '/content/drive/Othercomputers/MacBook_Air/path-finding-rl/data/model.pt')

    # 모든 에피소드 종료 후 결과 디스플레이 및 q_target 업데이트
    cur_time = datetime.now(tz)
    simple_cur_time = cur_time.strftime("%H:%M:%S")
    print('▶ Episode #', n_epi, 'start time:', simple_cur_time, end='→')

    q_target.load_state_dict(q.state_dict())

    print("n_episode :{}, score : {:.1f}, n_buffer : {}, eps : {:.1f}%".format(n_epi, score/print_interval, memory.size(), epsilon*100))
    
    # 학습된 결과 저장
    # torch.save(q, '/content/drive/MyDrive/aiffelthon/data/model.pt')
    torch.save(q, '/content/drive/Othercomputers/MacBook_Air/path-finding-rl/data/model.pt')

In [9]:
#train 모드 실행
main()

data/factory_order_train.csv used
len(env.files): 39999
[[ 100.  100.  100. -100.  100.  100.  100. -100. -100.]
 [   1.    1.    1.    1.    1.    1.    1.    1.    1.]
 [ 100.    1.    1.    1.    1.    1.    1.    1.  100.]
 [ 100.    1.    0.    1.    0.    1.    0.    1.  100.]
 [ 100.    1.    0.    1.    0.    1.    0.    1.  100.]
 [ 100.    1.    0.    1.    0.    1.    0.    1.  100.]
 [   1.    1.    0.    1.    0.    1.    0.    1.    1.]
 [   1.    1.    1.    1.    1.    1.    1.    1.    1.]
 [   1.    1.    1.    1.    1.    1.    1.    1.    1.]
 [   0.    0.    0.    0.   -5.    0.    0.    0.    0.]]
[[ 100. -100.  100.  100.  100.  100.  100.  100. -100.]
 [   1.    1.    1.    1.    1.    1.    1.    1.    1.]
 [ 100.    1.    1.    1.    1.    1.    1.    1.  100.]
 [ 100.    1.    0.    1.    0.    1.    0.    1. -100.]
 [ 100.    1.    0.    1.    0.    1.    0.    1. -100.]
 [ 100.    1.    0.    1.    0.    1.    0.    1. -100.]
 [   1.    1.    0.    1.    0.

RuntimeError: ignored

In [ ]:
## test 메서드 추가 #################################################################################
def test():
    # model = torch.load('/content/drive/MyDrive/aiffelthon/data/model.pt')
    model = torch.load('/content/drive/Othercomputers/MacBook_Air/path-finding-rl/data/model.pt')
    train_mode = False  # False
    tz = pytz.timezone('Asia/Seoul')
    env = Simulator()
    q = Qnet()
    s_before = None
    print_interval = 1  # 
    score = 0.0

    for n_epi in range(len(env.files)): # range()의 인수로 len(env.files) 사용하면 됨 (=1225)
        env.reset(n_epi)
        print('env.local_target:', env.local_target)
        ############################################
        ## 인풋 정보(obs)에 경로 길이 추가
        # path_length = len(env.local_target)
        # print('path_length', path_length)
        # s_before = env.curloc
        # s_before.append(path_length)
        ############################################        
        s = np.array(s_before)
        done = False

        while not done:  # OB, 장애물, 최종 목표 도달시 종료
            #####################################################################################
            # (option) test 경우 추가: 출발점에서는 무조건 위로 올라간다 (action=0)
            x, y, _ = s
            if [x,y] == [9,4]:
                a = 0
            else:
                a = q.test_action(torch.from_numpy(s).float())  # test_action 수행
            #####################################################################################
            #a = q.test_action(torch.from_numpy(s).float())  # test_action 수행
            #####################################################################################
            obs, r, cum_reward, done, goal_ob_reward = env.step(a)
            s_prime = np.array(env.curloc)
            s = s_prime
            score += r

        # print_interval (=1) 마다 현황 디스플레이하고 q_target 업데이트 및 score 초기화
        if n_epi % print_interval == 0 and n_epi != 0:
            cur_time = datetime.now(tz)
            simple_cur_time = cur_time.strftime("%H:%M:%S")
            print('▶ Episode #', n_epi, end=' → ')
            print('Score =', score, end='...')
            print('경로 길이:', len(env.actions), end=', ')
            if goal_ob_reward == 'finish':
                print('성공 여부: ', goal_ob_reward)
            else:
                print('성공 여부 : 실패', goal_ob_reward)
        
        score = 0.0
        print('→ pred_path:', env.actions)

    # 모든 에피소드 종료 후 결과 디스플레이
    # 코드 추가할 것!!!

In [ ]:
# test 모드 실행
train_mode = False
test()